# 作業 : 實作英文-德文翻譯機器人
***
## [作業目標]
用 PyTorch 實作一個英文-德文翻譯機器人

## [作業重點]
* 語言資料處理
* 使用 LSTM 建構 Encoder: EncoderLSTM
* 使用 LSTM 建構 Decoder: DecoderLSTM
* 搭建 Sequence to Sequence 模型: Seq2Seq
* 撰寫訓練函式
* 撰寫測試函式

## [問題]
在 Colab 上實際執行完這個範例後，請改用 BiLSTM 來建構 Encoder

## 引用需要的模組

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/D22_seq2seq_lab/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/D22_seq2seq_lab


In [3]:
!pip uninstall spacy -y
!pip install -U spacy

Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.8MB 239kB/s 
     |████████████████████████████████| 9.1MB 38.8MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 460kB 36.9MB/s 
     |████████████████████████████████| 1.1MB 44.5MB/s 
     |████████████████████████████████| 122kB 54.0MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=d0f6242ef58cde7fa1d336c5b422e2c725eba2fcb549bdf383132fa3272c797a
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.0.0
    Uninstalling smart-open-5.0.0:
      Successfully uninstalled smart-open-5.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
  

In [4]:
!python --version

Python 3.7.10


In [5]:
#!pip install torchtext==0.4
!pip install -U torchtext==0.4.0 #0.9.1

     |████████████████████████████████| 61kB 5.0MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [6]:
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
#from torchtext.experimental.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator
from torchtext.data import Field, BucketIterator

from torchtext.data.utils import get_tokenizer
#from torchtext.data.metrics import bleu_score

import numpy as np
import pandas as pd
import spacy
import random
# from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## 下載 spacy 英文/德文語料

In [7]:
!mkdir ./data
!mkdir ./data/multi30k
#!python -m spacy download en
!python -m spacy download en_core_web_sm
!ls ./data/multi30k -al
#spacy_english = spacy.load("en")
spacy_en = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/multi30k’: File exists
2021-05-27 21:22:59.005980: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 221kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 0
total 0


In [8]:
#!python -m spacy download de
!python -m spacy download de_core_news_sm

#spacy_de = spacy.load("de")
spacy_de = spacy.load("de_core_news_sm")


!ls ./data/multi30k -al

2021-05-27 21:23:08.520980: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 19.3MB 1.4MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 0


In [9]:
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# Sample Run
sample_text = 'I love machine learning'
print(tokenize_en(sample_text))

english = Field(tokenize=tokenize_en, lower=True,
                init_token='<sos>', eos_token='<eos>')
german = Field(tokenize=tokenize_de, lower=True,
               init_token='<sos>', eos_token='<eos>')

train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(english, german))
english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

print(f"Unique tokens in source (english) vocabulary: {len(english.vocab)}")
print(f"Unique tokens in target (german) vocabulary: {len(german.vocab)}")

['I', 'love', 'machine', 'learning']
Unique tokens in source (english) vocabulary: 5893
Unique tokens in target (german) vocabulary: 7853


In [10]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.']])


In [11]:
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device
)

In [12]:
len_eng_examples = []
len_ger_examples = []
for idx, data in enumerate(train_data):
    len_eng_examples.append(len(data.src))
    len_ger_examples.append(len(data.trg))
    if idx < 10:
        print('German - ', *data.src, ' Length - ', len(data.src))
        print('English - ', *data.trg, ' Length - ', len(data.trg))
        print()

print(f"Maximum Length of English sentence {max(len_eng_examples)} and German sentence {max(len_ger_examples)} in the dataset")
print(f"Minimum Length of English sentence {min(len_eng_examples)} and German sentence {min(len_ger_examples)} in the dataset")

German -  two young , white males are outside near many bushes .  Length -  11
English -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13

German -  several men in hard hats are operating a giant pulley system .  Length -  12
English -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8

German -  a little girl climbing into a wooden playhouse .  Length -  9
English -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10

German -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
English -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15

German -  two men are at the stove preparing food .  Length -  9
English -  zwei männer stehen am herd und bereiten essen zu .  Length -  10

German -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
English -  ein mann in grün hält eine gitarre , während der andere

In [13]:
data = next(iter(train_iterator))
temp_eng, temp_ger = data.src, data.trg
print('Shapes', data.src.shape, data.trg.shape, '\n')
print('English - ', *data.src, ' Length - ', len(data.src), '\n')
print('German - ', *data.trg, ' Length - ', len(data.trg))

Shapes torch.Size([22, 32]) torch.Size([25, 32]) 

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([  4,  16, 110,   4,   4,   4,  16,   4,   4,   4,   4,  21,   4,   4,
          4,   4,  21,   4,   8,  21,   4, 202,   4,   4,   4,   4,   4,   4,
          4,   4,   4,  16], device='cuda:0') tensor([  9,  63,  19, 188,   9,  34,  53, 174,  14,  64,  38, 324, 648,  24,
         14,   9, 145,  14,   7, 145,   9,  90,  24, 537, 168,  14,  24,  70,
        419, 683,   9, 115], device='cuda:0') tensor([3804,   17,   22,  105,    6,    6,  360,   13,   10,  239,   12,  327,
           6,  106,   13,    6,    9,    6,   39,   15,  151,   11,   34,  697,
          42,    6,    9,   55,   14,    9,   11,  127], device='cuda:0') tensor([   7,   22,   25,  140,   26,  731,   32,  163,  151,   10, 1398,   33,
           4,   34,    4,  602,   10,    4,    4, 3141,    4,  204,   13, 2764,
         4

In [14]:
temp_eng_idx = temp_eng.cpu().detach().numpy()
temp_ger_idx = temp_ger.cpu().detach().numpy()

In [15]:
df_eng_idx = pd.DataFrame(data=temp_eng_idx,
                          columns=[str('S_') + str(x + 1) for x in range(BATCH_SIZE)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,16,110,4,4,4,16,4,4,4,4,21,4,4,4,4,21,4,8,21,4,202,4,4,4,4,4,4,4,4,4,16
3,9,63,19,188,9,34,53,174,14,64,38,324,648,24,14,9,145,14,7,145,9,90,24,537,168,14,24,70,419,683,9,115
4,3804,17,22,105,6,6,360,13,10,239,12,327,6,106,13,6,9,6,39,15,151,11,34,697,42,6,9,55,14,9,11,127
5,7,22,25,140,26,731,32,163,151,10,1398,33,4,34,4,602,10,4,4,3141,4,204,13,2764,433,4,22,15,140,6,4,6
6,57,86,17,27,147,846,8,97,4,169,673,189,31,601,262,1045,119,590,9,1634,1407,711,535,4,9,420,4,22,4,4,55,1161
7,12,339,8,848,11,380,4,11,24,4,74,2309,1825,54,59,11,18,10,91,15,11,17,8,1356,6,695,109,4,90,25,17,3224
8,4,11,4,15,4,4,291,289,34,330,1216,454,11,2164,26,4,82,6,11,9,4002,345,27,18,4,22,67,194,1594,23,3604,1330
9,240,29,70,28,52,4632,108,2662,15,838,11,5486,25,69,2240,178,9,7,137,10,10,69,158,7,197,4,11,23,1208,1163,261,8


In [16]:
idx2word = {idx: word for idx, word in enumerate(english.vocab.itos)}
df_eng_word = pd.DataFrame(columns=[str('S_') + str(x + 1) for x in range(BATCH_SIZE)])
df_eng_word = df_eng_idx.replace(idx2word)
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,a,two,four,a,a,a,two,a,a,a,a,an,a,a,a,a,an,a,on,an,a,many,a,a,a,a,a,a,a,a,a,two
3,man,children,people,baseball,man,boy,little,guy,woman,person,group,african,skier,young,woman,man,old,woman,the,old,man,pink,young,martial,middle,woman,young,small,laughing,bald,man,older
4,wipes,are,wearing,player,in,in,guys,with,is,dancing,of,american,in,asian,with,in,man,in,street,",",carrying,and,boy,artist,-,in,man,child,woman,man,and,boys
5,the,wearing,white,holds,black,swim,sitting,long,carrying,is,youth,girl,a,boy,a,military,is,a,a,perhaps,a,purple,with,lands,aged,a,wearing,",",holds,in,a,in
6,outside,orange,are,his,pants,trunks,on,hair,a,doing,pass,who,red,leaps,very,camouflage,talking,bikini,man,homeless,broom,balloons,flower,a,man,public,a,wearing,a,a,child,hooded
7,of,vest,on,bat,and,does,a,and,young,a,some,had,snowsuit,for,large,and,to,is,sits,",",and,are,on,blow,in,place,top,a,pink,white,are,sweatshirts
8,a,and,a,",",a,a,brick,big,boy,dance,vehicles,just,and,joy,black,a,another,in,and,man,dustpan,crowded,his,to,a,wearing,hat,striped,star,shirt,purchasing,lay
9,window,blue,small,while,green,backflip,wall,belly,",",move,and,received,white,into,wig,dark,man,the,plays,is,is,into,face,the,suit,a,and,shirt,shaped,kneels,ice,on


## 用 LSTM 搭建 Encoder 類別: EncoderLSTM

In [17]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop_rate):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, 
                            bidirectional=True, dropout=drop_rate)
        self.dropout = nn.Dropout(drop_rate)

    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)
        hidden_state = hidden_state.view(self.num_layers, -1, self.hidden_size * 2)
        cell_state = cell_state.view(self.num_layers, -1, self.hidden_size * 2)

        return hidden_state, cell_state

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 256
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (embedding): Embedding(5893, 300)
  (LSTM): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


## 用 LSTM 搭建 decoder 類別: DecoderLSTM

In [18]:
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size,
                 num_layers, output_size, drop_rate):
        super(DecoderLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.LSTM = nn.LSTM(embedding_size, hidden_size,
                            num_layers, dropout=drop_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(drop_rate)

    def forward(self, x, hidden_state, cell_state):
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state

input_size_decoder = len(german.vocab)
decoder_embedding_size = 300
hidden_size = 512
num_layers = 2
decoder_dropout = 0.5
output_size = len(german.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, output_size, decoder_dropout).to(device)
print(decoder_lstm)

DecoderLSTM(
  (embedding): Embedding(7853, 300)
  (LSTM): LSTM(300, 512, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=512, out_features=7853, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [19]:
batch = next(iter(train_iterator))
print(batch.src.shape)
print(batch.trg.shape)

x = batch.trg[1]
print(x)

torch.Size([17, 32])
torch.Size([20, 32])
tensor([   5,   15,   43,    5, 1046,    8,   18,    5,    8,    8,   18,    5,
           8,    5,    5,    5,    8,    5,    8,    5,    5,    5,    5,   18,
           5,   18,    8,    8,    5,    5,   18,    5], device='cuda:0')


## Seq2Seq(Sequence to Sequence)類別

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, Encoder_LSTM, Decoder_LSTM):
        super(Seq2Seq, self).__init__()
        self.Encoder_LSTM = Encoder_LSTM
        self.Decoder_LSTM = Decoder_LSTM

    def forward(self, source, target, tfr=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(german.vocab)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden_state, cell_state = self.Encoder_LSTM(source)

        x = target[0]   # Trigger token <SOS>
        for i in range(1, target_len):
            output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1)   # 0th dimension is batch size, 1st dimension is word embedding size
            # Either pass the next word correctly from the dataset or use the earlier predicted word
            x = target[i] if random.random() < tfr else best_guess

        return outputs

In [21]:
# Hyperparameters
learning_rate = 0.001
step = 0
writer = SummaryWriter(os.path.join('runs', 'loss_plot'))

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [22]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (embedding): Embedding(5893, 300)
    (LSTM): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (Decoder_LSTM): DecoderLSTM(
    (embedding): Embedding(7853, 300)
    (LSTM): LSTM(300, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=7853, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [23]:
def translate(model, sentence, english, german, device, max_length=50):
    if type(sentence) == str:
        tokens = tokenize_en(sentence)
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [german.vocab.stoi['<sos>']]
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == german.vocab.stoi['<eos>']:
            break

    translated_sentence = [german.vocab.itos[idx] for idx in outputs]
    
    return translated_sentence[1:]   # ignore <sos> token

In [24]:
def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving\n')
    state = {'model': model, 'best_loss': best_loss, 'epoch': epoch,
             'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict()}
    torch.save(state, 'checkpoint-NMT')
    torch.save(model.state_dict(), 'checkpoint-NMT-SD')

In [25]:
# 用來評估模型的函式: bleu
def bleu(data, model, english, german, device):
    targets, outputs = [], []
    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate(model, src, english, german, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [26]:
epoch_loss = 0.0
num_epochs = 30
best_loss = 999999
best_epoch = -1
sentence = 'a man in a blue shirt is standing on a ladder and cleaning a window'
ts = []

for epoch in range(num_epochs):
    model.eval()
    print(f"Epoch - {epoch + 1} / {num_epochs}")
    translated_sentence = translate(model, sentence, english, german, device, max_length=50)
    print(f"Translated example sentence: \n {translated_sentence}")
    ts.append(translated_sentence)

    model.train()
    for batch_idx, batch in enumerate(train_iterator):
        input = batch.src.to(device)
        target = batch.trg.to(device)

        # Pass the input and target for model's forward method
        output = model(input, target)
        output = output[1:].view(-1, output.shape[2])
        target = target[1:].view(-1)

        # Calculate the loss value for every epoch
        loss = criterion(output, target)
        
        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 5.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        # Update the weights values using the gradients we calculated using bp 
        optimizer.step()
        step += 1
        epoch_loss += loss.item()
        writer.add_scalar('Training loss', loss, global_step=step)

    if epoch_loss < best_loss:
        best_epoch = epoch
        best_loss = epoch_loss
        checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
        if (epoch - best_epoch >= 10):
            print('no improvement in 10 epochs, break')
            break
    print(f"Epoch_Loss - {loss.item()}\n")

print(epoch_loss / len(train_iterator))

Epoch - 1 / 30
Translated example sentence: 
 ['zugs', 'zugs', 'zugs', 'geldautomaten', 'zugs', 'geldautomaten', 'gürteln', 'gürteln', 'dreckigen', 'blondine', 'blondine', 'zugedeckt', 'zugedeckt', 'aktiv', 'broschüre', 'broschüre', 'broschüre', 'skilift', 'begriff', 'begriff', 'stativ', 'unteren', 'zusammengebauten', 'künstlern', 'bearbeiten', 'künstlern', 'softdrink', 'ansieht', 'ansieht', 'ansieht', 'pirat', 'orangensaft', 'vorn', 'schönen', 'grillt', 'grillt', 'polizeibeamte', 'cheerleaders', 'dreiköpfige', 'rutschen', 'rutschen', 'monopoly-spiel', 'knipst', 'träg', 'träg', 'kunstprojekt', 'beugen', 'feuerwehrauto', 'feuerwehrauto', 'violette']
saving

Epoch_Loss - 4.365269184112549

Epoch - 2 / 30
Translated example sentence: 
 ['ein', 'mann', 'in', 'einem', 'einem', 'hemd', 'und', 'einem', 'einem', '<unk>', '.', '<eos>']
Epoch_Loss - 4.041811943054199

Epoch - 3 / 30
Translated example sentence: 
 ['ein', 'mann', 'in', 'einem', 'einem', 'hemd', 'und', 'einem', 'einem', '<unk>', '